In [11]:
dataset = [
    ("The dog ate the apple".lower().split(), ["D", "N", "V", "D", "N"]),
    ("Everybody read that book".lower().split(), ["N", "V", "D", "N"]),
    ("Trapp is sleeping".lower().split(), ["N", "V", "V"]),
    ("Everybody ate the apple".lower().split(), ["N", "V", "D", "N"]),
    ("Cats are good".lower().split(), ["N", "V", "D"]),
    ("Dogs are not as good as cats".lower().split(),["N", "V", "D", "D", "D", "D", "N"]),
    ("Dogs eat dog food".lower().split(), ["N", "V", "N", "N"]),
    ("Watermelon is the best food".lower().split(), ["N", "V", "D", "D", "N"]),
    ("I want a milkshake right now".lower().split(), ["N", "V", "D", "N", "D", "D"]),
    ("I have too much homework".lower().split(), ["N", "V", "D", "D", "N"]),
    ("Zoom won't work".lower().split(), ["N", "D", "V"]),
    ("Pie also sounds good".lower().split(), ["N", "D", "V", "D"]),
    ("The college is having the department fair this Friday".lower().split(),["D", "N", "V", "V", "D", "N", "N", "D", "N"]),
    ("Research interests span many areas".lower().split(), ["N", "N", "V", "D", "N"]),
    ("Alex is finishing his Ph.D".lower().split(), ["N", "V", "V", "D", "N"]),
    ("She is the author".lower().split(), ["N", "V", "D", "N"]),
    ("It is almost the end of the semester".lower().split(), ["N", "V", "D", "D", "N", "D", "D", "N"]),
    ("Blue is a color".lower().split(), ["N", "V", "D", "N"]),
    ("They wrote a book".lower().split(), ["N", "V", "D", "N"]),
    ("The syrup covers the pancake".lower().split(), ["D", "N", "V", "D", "N"]),
    ("Harrison has these teeth".lower().split(), ["N", "V", "D", "N"]),
    ("The numbers are fractions".lower().split(), ["D", "N", "V", "N"]),
    ("Yesterday happened".lower().split(), ["N", "V"]),
    ("Caramel is sweet".lower().split(), ["N", "V", "D"]),
    ("Computers use electricity".lower().split(), ["N", "V", "N"]),
    ("Gold is a valuable thing".lower().split(), ["N", "V", "D", "D", "N"]),
    ("This extension cord helps".lower().split(), ["D", "D", "N", "V"]),
    ("It works on my machine".lower().split(), ["N", "V", "D", "D", "N"]),
    ("We have the words".lower().split(), ["N", "V", "D", "N"]),
    ("Trapp is a dog".lower().split(), ["N", "V", "D", "N"]),
    ("This is a computer".lower().split(), ["N", "V", "D", "N"]),
    ("I love lamps".lower().split(), ["N", "V", "N"]),
    ("I walked outside".lower().split(), ["N", "V", "N"]),
    ("You never bike home".lower().split(), ["N", "D", "V", "N"]),
    ("You are a wizard Harry".lower().split(), ["N", "V", "D", "N", "N"]),
    ("Trapp ate the shoe".lower().split(), ["N", "V", "D", "N"]),
    ("Jett failed his test".lower().split(), ["N", "V", "D", "N"]),
    ("Alice won the game".lower().split(), ["N", "V", "D", "N"]),
    ("The class lasted a semester".lower().split(), ["D", "N", "V", "D", "N"]),
    ("The tree had a branch".lower().split(), ["D", "N", "V", "D", "N"]),
    ("I ran a race".lower().split(), ["N", "V", "D", "N"]),
    ("The dog barked".lower().split(), ["D", "N", "V"]),
    ("Toby hit the wall".lower().split(), ["N", "V", "D", "N"]),
    ("Zayn ate an apple".lower().split(), ["N", "V", "D", "N"]),
    ("The cat fought the dog".lower().split(), ["D", "N", "V", "D", "N"]),
    ("I got an A".lower().split(), ["N", "V", "D", "N"]),
    ("The A hurt".lower().split(), ["D", "N", "V"]),
    ("I jump".lower().split(), ["N", "V"]),
    ("I drank a yerb".lower().split(), ["N", "V", "D", "N"]),
    ("The snake ate a fruit".lower().split(), ["D", "N", "V", "D", "N"]),
    ("I played the game".lower().split(), ["N", "V", "D", "N"]),
    ("I watched a movie".lower().split(), ["N", "V", "D", "N"]),
    ("Clark fixed the audio".lower().split(), ["N", "V", "D", "N"]),
    ("I went to Frary".lower().split(), ["N", "V", "D", "N"]),
    ("I go to Pomona".lower().split(), ["N", "V", "D", "N"]),
    ("Food are friends not fish".lower().split(), ["N", "V", "N", "D", "N"]),
    ("You are reading this".lower().split(), ["N", "V", "D", "N"]),
    ("Wonderland protocol is amazing".lower().split(), ["D", "N", "V", "D"]),
    ("This is a sentence".lower().split(), ["D", "V", "D", "N"]),
    ("I should be doing homework".lower().split(), ["N", "V", "V", "V", "N"]),
    ("Computers are tools".lower().split(), ["N", "V", "N"]),
    ("The whale swims".lower().split(), ["D", "N", "V"]),
    ("A cup is filled".lower().split(), ["D", "N", "V", "V"]),
    ("This is a cat".lower().split(), ["D", "V", "D", "N"]),
    ("These are trees".lower().split(), ["D", "V", "N"]),
    ("The cat is the teacher".lower().split(), ["D", "N", "V", "D", "N"]),
    ("I ate food today".lower().split(), ["N", "V", "N", "N"]),
    ("I am a human".lower().split(), ["N", "V", "D", "N"]),
    ("The cat sleeps".lower().split(), ["D", "N", "V"]),
    ("Whales are mammals".lower().split(), ["N", "V", "N"]),
    ("I like turtles".lower().split(), ["N", "V", "N"]),
    ("A shark ate me".lower().split(), ["D", "N", "V", "N"]),
    ("There are mirrors".lower().split(), ["D", "V", "N"]),
    ("The bus spins".lower().split(), ["D", "N", "V"]),
    ("Computers are machines".lower().split(), ["N", "V", "N"])
]

In [12]:
import torch
from random import shuffle

In [18]:
word_to_index = {}
tag_to_index = {}

total_words = 0
total_tags = 0

tag_list = []

for words, tags in dataset:
    
    assert len(words) == len(tags)
    
    total_words += len(words)
    
    for word in words:
        if word not in word_to_index:
            word_to_index[word] = len(word_to_index)
    
    total_tags += len(tags)
    
    for tag in tags:
        if tag not in tag_to_index:
            tag_to_index[tag] = len(tag_to_index)

In [19]:
print("       Vocabulary Indices")
print("-------------------------------")

for word in sorted(word_to_index):
    print(f"{word:>14} => {word_to_index[word]:>2}")

print("\nTotal number of words:", total_words)
print("Number of unique words:", len(word_to_index))

       Vocabulary Indices
-------------------------------
             a => 23
          alex => 48
         alice => 106
        almost => 55
          also => 35
            am => 160
       amazing => 147
            an => 121
         apple =>  3
           are => 12
         areas => 47
            as => 16
           ate =>  2
         audio => 136
        author => 53
        barked => 116
            be => 150
          best => 20
          bike => 98
          blue => 59
          book =>  7
        branch => 113
           bus => 171
       caramel => 74
           cat => 122
          cats => 11
         clark => 134
         class => 109
       college => 37
         color => 60
      computer => 91
     computers => 76
          cord => 83
        covers => 64
           cup => 155
    department => 39
           dog =>  1
          dogs => 14
         doing => 151
         drank => 127
           eat => 17
   electricity => 78
           end => 56
     everybody =>  4
   

In [20]:
print("Tag Indices")
print("-----------")

for tag, index in tag_to_index.items():
    print(f"  {tag} => {index}")

print("\nTotal number of tags:", total_tags)
print("Number of unique tags:", len(tag_to_index))

Tag Indices
-----------
  D => 0
  N => 1
  V => 2

Total number of tags: 308
Number of unique tags: 3


In [30]:
def convert_words_to_index_tensor(words, mapping):
    indices = [mapping[w] for w in words]
    return torch.tensor(indices, dtype=torch.long)

In [22]:
vocab_size = len(word_to_index)
embed_dim = 6 # Hyperparameter
embed_layer = torch.nn.Embedding(vocab_size, embed_dim)

In [36]:
# i = torch.tensor([word_to_index["the"], word_to_index["dog"]])
indices = convert_words_to_index_tensor("The dog ate the apple".lower().split(), word_to_index)
embed_output = embed_layer(indices)
embed_output.shape, embed_output

(torch.Size([5, 6]),
 tensor([[ 1.9823, -1.7323,  0.3668, -1.7455, -0.3420,  1.3482],
         [-0.5125, -0.9508, -0.3973, -0.3678,  0.5798, -0.4140],
         [ 1.2710,  0.2771, -0.1242, -0.3522, -1.1643,  0.3411],
         [ 1.9823, -1.7323,  0.3668, -1.7455, -0.3420,  1.3482],
         [ 0.4866, -0.8927, -0.8811, -1.3986,  1.1610,  0.7400]],
        grad_fn=<EmbeddingBackward>))

In [51]:
hidden_dim = 10
lstm_layer = torch.nn.LSTM(embed_dim, hidden_dim)

In [54]:
# (L, N, E)
lstm_output, _ = lstm_layer(embed_output.unsqueeze(1))
lstm_output.shape

torch.Size([5, 1, 10])

In [56]:
linear_layer = torch.nn.Linear(hidden_dim, len(tag_to_index))

In [59]:
linear_output = linear_layer(lstm_output)
linear_output.shape, linear_output

(torch.Size([5, 1, 3]),
 tensor([[[-0.1569, -0.0117, -0.1078]],
 
         [[-0.1784, -0.0688, -0.1352]],
 
         [[-0.2861, -0.0483, -0.1489]],
 
         [[-0.1702,  0.0042, -0.0940]],
 
         [[-0.1342, -0.0075, -0.0763]]], grad_fn=<AddBackward0>))

In [91]:
class POS_LSTM(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, tag_size):
        super().__init__()
        self.embed = torch.nn.Embedding(vocab_size, embed_dim)
        self.lstm = torch.nn.LSTM(embed_dim, hidden_dim)
        self.linear = torch.nn.Linear(hidden_dim, tag_size)
        
    def forward(self, X):
        X = self.embed(X)
        X, _ = self.lstm(X.unsqueeze(1))
        return self.linear(X)

In [92]:
model = POS_LSTM(vocab_size, embed_dim, hidden_dim, len(tag_to_index))

In [93]:
indices.shape

torch.Size([5])

In [94]:
model(indices)

torch.Size([5])
torch.Size([5, 6])
torch.Size([5, 1, 6])
torch.Size([5, 1, 10])


tensor([[[-0.0752, -0.2781, -0.0217]],

        [[-0.0524, -0.2907,  0.0225]],

        [[-0.0524, -0.2682,  0.0389]],

        [[-0.0578, -0.3125,  0.0060]],

        [[-0.0263, -0.3917, -0.0846]]], grad_fn=<AddBackward0>)